In [51]:
# Import the Libraries
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pickle

In [52]:
# Load the dataset
path = "../EDA/COPD_engineered_data.csv"
data = pd.read_csv(path)
data.head()

,Age,Gender,Smoking_Status,Family_History_COPD,BMI,Location,Air_Pollution_Level,COPD_Diagnosis,Age_Category,BMI_Category,Pollution_Risk_Score,Smoking_Status_Encoded,Smoking_Pollution_interaction
0,31,Male,Former,1,27.56,Lalitpur,84,0,30-39,Overweight,0,0.5,42.0
1,60,Male,Never,0,30.30,Pokhara,131,0,60-69,Obessity,0,0.0,0.0
2,33,Male,Former,1,28.45,Pokhara,123,0,30-39,Overweight,0,0.5,61.5
3,36,Female,Current,0,27.49,Kathmandu,253,1,30-39,Overweight,1,NaN,NaN
4,58,Male,Never,0,25.49,Pokhara,117,0,50-59,Overweight,0,0.0,0.0


In [53]:
print(data.isnull().sum())
data1 = data.dropna()



Age                                0
Gender                             0
Smoking_Status                     0
Family_History_COPD                0
BMI                                0
Location                           0
Air_Pollution_Level                0
COPD_Diagnosis                     0
Age_Category                      20
BMI_Category                       0
Pollution_Risk_Score               0
Smoking_Status_Encoded           357
Smoking_Pollution_interaction    357
dtype: int64


In [19]:
# Define the features and target variable
X = data1.drop(columns=['COPD_Diagnosis'])
y = data1['COPD_Diagnosis']

In [20]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train_encoded = pd.get_dummies(X_train) 
X_test_encoded = pd.get_dummies(X_test)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

In [22]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}
# Train and save models
for name, model in models.items():
    # Train model with encoded data
    model.fit(X_train_encoded, y_train)

    # Save the model
    model_filename = f'{name.replace(" ", "_")}.pkl'
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"{name} model trained and saved.")

print("Model training completed.\n")
# Evaluate the models
for name, model in models.items():
    model_filename = f'{name.replace(" ", "_")}.pkl'
    with open(model_filename, 'rb') as f:
        model = pickle.load(f)

    # Make predictions
    y_pred = model.predict(X_test_encoded)

    # Print evaluation metrics
    print(f"\n{name} Evaluation: ")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))


Logistic Regression model trained and saved.
Decision Tree model trained and saved.
Random Forest model trained and saved.
Model training completed.


Logistic Regression Evaluation: 
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97       114
           1       0.83      0.42      0.56        12

    accuracy                           0.94       126
   macro avg       0.89      0.70      0.76       126
weighted avg       0.93      0.94      0.93       126


Decision Tree Evaluation: 
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       114
           1       0.67      0.50      0.57        12

    accuracy                           0.93       126
   macro avg       0.81      0.74      0.77       126
weighted avg       0.92      0.93      0.92       126


Random Forest Evaluation: 
Classification Report:
              precision    recall  f1

In [23]:
# Define the parameter grid for Random forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(), 
                           param_grid=param_grid, 
                           cv=5, 
                           n_jobs=-1,  
                           scoring='accuracy')

grid_search.fit(X_train_encoded, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [28]:
# Best parameters
print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

Best Parameters: {'max_depth': 30, 'min_samples_split': 10, 'n_estimators': 200}


In [30]:
# Save the best model
with open('Best_Random_Forest_Model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model refinement completed and best model saved")

Model refinement completed and best model saved
